In [ ]:
import pandas as pd
import os
import re
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Chemin racine des données
base_dir = "data"
all_data = []

for semaine in os.listdir(base_dir):
    semaine_path = os.path.join(base_dir, semaine)
    if os.path.isdir(semaine_path):
        for file in os.listdir(semaine_path):
            if file.endswith(".csv") and "Workout" in file:
                file_path = os.path.join(semaine_path, file)
                try:
                    df = pd.read_csv(file_path, encoding="utf-8")
                except UnicodeDecodeError:
                    try:
                        df = pd.read_csv(file_path, encoding="ISO-8859-1")
                    except Exception as e:
                        print(f"Erreur de lecture pour {file_path} : {e}")
                        continue
                df["semaine"] = semaine
                df["source"] = file
                all_data.append(df)

if all_data:
    full_df = pd.concat(all_data, ignore_index=True)
    full_df["date"] = pd.to_datetime(full_df["date"], errors='coerce')

    def extract_weight(value):
        if pd.isna(value):
            return None
        match = re.search(r'\d+\.?\d*', str(value))
        return float(match.group()) if match else None

    full_df["poids_num"] = full_df["poids"].apply(extract_weight)

    # Dropdown interactif
    exercise_dropdown = widgets.Dropdown(
        options=sorted(full_df["exercices"].dropna().unique()),
        description='Exercice:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    def plot_exercise_progression(exercise):
        filtered = full_df[full_df["exercices"] == exercise]
        fig = px.line(filtered, x='date', y='poids_num', markers=True,
                      title=f"Progression du poids pour : {exercise}",
                      labels={"poids_num": "Poids (kg)", "date": "Date"})
        fig.update_layout(xaxis_title="Date", yaxis_title="Poids utilisé (kg)")
        fig.show()

    display(widgets.interact(plot_exercise_progression, exercise=exercise_dropdown))
else:
    print("❌ Aucun fichier CSV trouvé ou lisible dans les sous-dossiers.")



interactive(children=(Dropdown(description='Exercice:', layout=Layout(width='50%'), options=('Curl biceps banc…

<function __main__.plot_exercise_progression(exercise)>